In [1]:
import zipfile
import os

from keras import utils
import tensorflow as tf

2024-08-19 12:03:01.098106: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-19 12:03:01.098164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-19 12:03:01.117032: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-19 12:03:01.163050: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-19 12:03:02.928513: W tensorflow/comp

In [5]:
import os
import zipfile

def unzip_to_cwd(zip_filepath, target_folder='train'):
    current_dir = os.getcwd()
    target_folder_path = os.path.join(current_dir, target_folder)

    if not os.path.exists(target_folder_path):
        os.makedirs(target_folder_path)

    with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
        zip_ref.extractall(target_folder_path)

zip_filepath = 'images_zipped.zip'
unzip_to_cwd(zip_filepath)

In [2]:
path_to_data = './train'

In [5]:
train_data=utils.image_dataset_from_directory(
    path_to_data,
    labels="inferred",
    label_mode="int",
    validation_split=0.1,
    subset="training",
    shuffle=True,
    color_mode="rgb",
    image_size=(256,256),
    batch_size=64,
    seed=40,
)

Found 1129 files belonging to 10 classes.
Using 1017 files for training.


In [6]:
val_data=utils.image_dataset_from_directory(
    path_to_data,
    labels="inferred",
    label_mode="int",
    validation_split=0.1,
    subset="validation",
    color_mode="rgb",
    image_size=(256,256),
    batch_size=64,
    seed=40,
)

Found 1129 files belonging to 10 classes.
Using 112 files for validation.


In [8]:
for images,lables in train_data.take(1):
  print('Amount of train images: ', images.shape)
  print('Amount of train labels: ', lables.shape)

Amount of train images:  (64, 256, 256, 3)
Amount of train labels:  (64,)


In [9]:
for images,lables in val_data.take(1):
  print('Amount of validation images: ', images.shape)
  print('Amount of validation labels: ', lables.shape)

Amount of validation images:  (64, 256, 256, 3)
Amount of validation labels:  (64,)


In [10]:
classes=train_data.class_names
print('The following classes are in the dataset:')
print(classes)

The following classes are in the dataset:
['Avulsion fracture', 'Comminuted fracture', 'Fracture Dislocation', 'Greenstick fracture', 'Hairline Fracture', 'Impacted fracture', 'Longitudinal fracture', 'Oblique fracture', 'Pathological fracture', 'Spiral Fracture']


In [11]:
def normalize(image, label):
  return image/255.0, label

In [12]:
train_data = train_data.map(normalize)
val_data= val_data.map(normalize)

In [13]:
train_x=[]
train_y=[]

for image,label in train_data:
   train_x.append(image)
   train_y.append(label)

train_x = tf.concat(train_x, axis=0)
train_y = tf.concat(train_y, axis=0)

2024-08-19 09:31:34.549921: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 799801344 exceeds 10% of free system memory.


In [14]:
val_x=[]
val_y=[]

for image,label in train_data:
   val_x.append(image)
   val_y.append(label)

val_x = tf.concat(val_x, axis=0)
val_y = tf.concat(val_y, axis=0)

2024-08-19 09:31:36.645161: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 799801344 exceeds 10% of free system memory.


In [15]:
num_classes = 10
train_y = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)
val_y = tf.keras.utils.to_categorical(val_y, num_classes=num_classes)

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(Conv2D(60, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(120, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(40, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_x, train_y, batch_size=10, epochs=5,
          validation_data=(val_x,val_y))

In [ ]:
loss, accuracy = model.evaluate(val_x,val_y)
print('Test accuracy:', accuracy)

In [ ]:
model.save('fracture_model.h5')